## [Chapter 15: Foundation Models and Emerging Search Paradigms]
## Generative AI and the Search Frontier

In [3]:
import html
import json
import pickle
import sys
import warnings
sys.path.append("..")

import pandas
import spacy
from aips import *
from aips.spark import create_view_from_collection
from IPython.display import HTML, display
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import ArrayType, FloatType
import numpy

warnings.filterwarnings("ignore") #Some operations warn inside a loop, we'll only need to see the first warning

engine = get_engine()
outdoors_collection = engine.get_collection("outdoors")
spark = SparkSession.builder.appName("AIPS").getOrCreate()

**Note**: This notebook depends upon the Outdoors dataset. If you have any issues, please rerun the [Setting up the Outdoors Dataset](../ch13/1.setting-up-the-outdoors-dataset.ipynb) notebook.

## Mocked Generative Output

For this notebook, we mock responses from a large generative model.

We encourage you to explore connecting to a generative model API of your choice, and comparing results!

In [2]:
import html
mockedGenerativeResponses = pandas.read_csv("mockedGenerativeResponses.csv")
def get_generative_response(prompt):
    prompt = prompt.replace("""
""", "\r\n")
    prompt=prompt.replace("\n", "\n") #marshall new lines into consistent format
    response = mockedGenerativeResponses.loc[mockedGenerativeResponses["prompt"] == prompt, "response"].values
    if len(response) > 0:
        output = f'''
        <div style="overflow: auto; margin-bottom: 10px;">
          <h3 style="float: left; width: 100px; margin-right: 10px;">Query:</h3>
          <p style="overflow: hidden; margin-left: 110px;"><pre>{html.escape(prompt)}<pre></p>
        </div>

        <div style="overflow: auto; margin-bottom: 10px;">
          <h3 style="float: left; width: 100px; margin-right: 10px;">Response:</h3>
          <p style="overflow: hidden; margin-left: 110px;"><pre>{html.escape(response[0])}</pre></p>
        </div>
        '''
        display(HTML(output))
        return response[0]
    else:
        print(f"\n\nSorry! your prompt does not have a mocked value.")
        return ""

## Retriever from Listing 14.6

See [../ch14/2.ch14-question-answering-CPU-data-preparation.ipynb#Listing-14.6](../ch14/2.ch14-question-answering-CPU-data-preparation.ipynb#Listing-14.6)

In [3]:
nlp = spacy.load("en_core_web_sm")
nlp.remove_pipe("ner")
nlp.add_pipe("merge_noun_chunks")
determiners = "all an another any both del each either every half la many much nary neither no some such that the them these this those".split(" ")
def get_query_from_question(question):
    query = []
    doc = nlp(question)
    for tok in doc:
        if tok.pos_ in ["NOUN","VERB"]:
            query.append(tok.text)
    if not len(query):
        query = [question]
    query = " ".join(query)
    for d in determiners:
        query = query.replace(" "+d+" ","")
    return query

In [4]:
def retriever(question):
    contexts={"id":[],"question":[],"context":[],"url":[]}
    query = get_query_from_question(question)
    request = {
        "query": query,
        "query_fields": ["body"],
        "return_fields": ["id", "url", "body"],
        "filters": [("post_type", "answer")],
        "limit": 5
    }
    docs = outdoors_collection.search(**request)["docs"]
    for doc in docs:
        contexts["id"].append(doc["id"])
        contexts["url"].append(doc["url"])
        contexts["question"].append(question)
        contexts["context"].append(doc["body"])
    return pandas.DataFrame(contexts)

## Listing 15.1

In [5]:
r = get_generative_response("What is a unicorn?")

## Listing 15.2

In [6]:
r = get_generative_response("""You are a chatbot named AIPS chat. 
You have a Ph.D. in biology. 
What is a unicorn?""")

## Listing 15.3

## Listing 15.4

(The prompt and responses for Listings 15.3 and 15.4 are both represented in one cell)

In [7]:
r = get_generative_response("""Web search results:

[1] "A large language model, or LLM, is a deep learning algorithm that 
can recognize, summarize, translate, predict and generate text and other 
content based on knowledge gained from massive datasets."
URL: https://blogs.nvidia.com/blog/2023/01/26/what-are-large-language-models-used-for/

[2] A large language model (LLM) is a language model consisting of a 
neural network with many parameters (typically billions of weights or 
more), trained on large quantities of unlabeled text using 
self-supervised learning. LLMs emerged around 2018 and perform well at a 
wide variety of tasks. This has shifted the focus of natural language 
processing research away from the previous paradigm of training 
specialized supervised models for specific tasks.
URL: https://en.wikipedia.org/wiki/Large_language_model

[3] "In this guide, we'll discuss everything you need to know about 
Large Language Models (LLMs), including key terms, algorithms, 
fine-tuning, and more. As ChatGPT has taken the internet by storm crossing 
1 million users in its first 5 days, you may be wondering what machine 
learning algorithm is running under the hood. While ChatGPT uses a 
specific type of reinforcement learning called "Reinforcement Learning 
from Human Feedback (RLHF)", at a high level it is an example of a 
Large Language Model (LLM)."
URL: https://www.mlq.ai/what-is-a-large-language-model-llm/

Instructions: Using the provided web search results, write a comprehensive 
reply to the given query. Make sure to cite results using 
[[number](URL)] notation after the reference. If the provided search 
results refer to multiple subjects with the same name, write separate 
answers for each subject.

Query: What is a large language model?""")

## Listing 15.5

In [8]:
r = get_generative_response("""Web search results:

[1] "A large language model, or LLM, is a deep learning algorithm that 
can recognize, summarize, translate, predict and generate text and other 
content based on knowledge gained from massive datasets."
URL: https://blogs.nvidia.com/blog/2023/01/26/what-are-large-language-models-used-for/

[2] A large language model (LLM) is a language model consisting of a 
neural network with many parameters (typically billions of weights or 
more), trained on large quantities of unlabeled text using 
self-supervised learning. LLMs emerged around 2018 and perform well at a 
wide variety of tasks. This has shifted the focus of natural language 
processing research away from the previous paradigm of training 
specialized supervised models for specific tasks.
URL: https://en.wikipedia.org/wiki/Large_language_model

[3] "In this guide, we'll discuss everything you need to know about 
Large Language Models (LLMs), including key terms, algorithms, 
fine-tuning, and more. As ChatGPT has taken the internet by storm crossing 
1 million users in its first 5 days, you may be wondering what machine 
learning algorithm is running under the hood. While ChatGPT uses a 
specific type of reinforcement learning called "Reinforcement Learning 
from Human Feedback (RLHF)", at a high level it is an example of a 
Large Language Model (LLM)."
URL: https://www.mlq.ai/what-is-a-large-language-model-llm/

Instructions: Using the provided web search results, write a comprehensive 
reply to the given query. Make sure to cite results using 
[[number](URL)] notation after the reference. If the provided search 
results refer to multiple subjects with the same name, write separate 
answers for each subject.

Query: What is a large language model?  Be concise.""")

## Listing 15.6

In [9]:
summarize_search_prompt = """What are some queries that should find the following documents?  List at least 5 unique queries, where these documents are better than others in an outdoors question and answer dataset.  Be concise and only output the list of queries, and a result number in the format [n] for the best result in the resultset.  Don't print a relevance summary at the end.

### Results:
{resultset}"""
print(summarize_search_prompt)

What are some queries that should find the following documents?  List at least 5 unique queries, where these documents are better than others in an outdoors question and answer dataset.  Be concise and only output the list of queries, and a result number in the format [n] for the best result in the resultset.  Don't print a relevance summary at the end.

### Results:
{resultset}


## Listing 15.7

In [10]:
example_contexts = retriever("What are minimalist shoes?")
resultset = [f"{idx}. {ctx}" for idx, ctx in enumerate(list(example_contexts[0:5]["context"]))]
print("\n".join(resultset))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Listing 15.8

In [ ]:
resultset_text = "\n".join(resultset)
resultset_prompt = summarize_search_prompt.replace("{resultset}", resultset_text)
generated_relevance_judgments = get_generative_response(resultset_prompt)

## Listing 15.9

In [ ]:
import re

def extract_pairwise_judgments(text, contexts):
    query_pattern = re.compile(r"\d+\.\s+(.*)")
    result_pattern = re.compile(r"\d+\.\s+\[(\d+)\]")    
    lines = text.split("\n")
    queries = []
    results = []
    for line in lines:
        query_match = query_pattern.match(line)
        result_match = result_pattern.match(line)
        if result_match:
            result_index = int(result_match.group(1))
            results.append(result_index)            
        elif query_match:
            query = query_match.group(1)
            queries.append(query)            
    output = [{"query": query, "relevant_document": contexts[result]["id"]}
              for query, result in zip(queries, results)]
    return output

## Listing 15.10

In [ ]:
resultset_contexts = example_contexts.to_dict("records")
extract_pairwise_judgments(generated_relevance_judgments, resultset_contexts)

## Listing 15.11

Article source: Alan Morrell, Democrat and Chronicle, May 29, 2023 https://www.democratandchronicle.com/story/money/business/2023/05/29/carvel-is-gone-from-rochester-what-happened-to-the-ice-cream-chain/70252613007/

In [ ]:
news_article_snippet = "Walter Silverman of Brighton owned one of the most successful local Carvel franchises, at East Ridge Road and Hudson Avenue in Irondequoit. He started working for Carvel in 1952. This is how it appeared in the late 1970s/early 1980s."
news_article_labelled = """<per>Walter Silverman</per> of <loc>Brighton</loc> owned one of the most successful local <org>Carvel</org> franchises, at <loc>East Ridge Road</loc> and <loc>Hudson Avenue</loc> in <loc>Irondequoit</loc>. He started working for <org>Carvel</org> in 1952. This is how it appeared in the late 1970s/early 1980s."""
print(news_article_labelled)

## Listing 15.12

In [ ]:
def extract_entities(text):
    entities = []
    pattern = r"<(per|loc|org)>(.*?)<\/(per|loc|org)>"
    matches = re.finditer(pattern, text)
    for match in matches:
        entity = {
            "label": match.group(1).upper(),
            "offset": [match.start(), match.end() - 1],
            "text": match.group(2)
        }
        entities.append(entity)
    return entities

In [ ]:
#Our manually labelled article
extract_entities(news_article_labelled)

## Listing 15.13

In [ ]:
entities_prompt = """For a given passage, please identify and mark the following entities: people with the tag '<per>', locations with the tag '<loc>', and organizations with the tag '<org>'. Please repeat the passage below with the appropriate markup.
### {text}"""
entities_prompt_news_article = entities_prompt.replace("{text}",news_article_snippet)
news_article_generated_labelled = get_generative_response(entities_prompt_news_article)
extract_entities(news_article_generated_labelled)

Up next: [Multi-modal Search](2.multi-modal-search.ipynb)